In [87]:
import sqlalchemy
import pandas as pd
#import json
import datetime

from sqlalchemy.orm import sessionmaker
import sqlite3
#import requests
import twint


In [88]:
DATABASE_LOCATION = "sqlite:////media/harsha/projects/data/imdb/movies_data.sqlite"
#Connecting to sqlite
engine = sqlalchemy.create_engine(DATABASE_LOCATION)
conn = sqlite3.connect('/media/harsha/projects/data/imdb/movies_data.sqlite')
#Creating a cursor object using the cursor() method
cursor = conn.cursor()

In [89]:
q_extract = """
SELECT ROWID as S_No
,user_id 
,username 
,tweet
,unix_time 
from tweets 
WHERE unix_time<(SELECT MIN(unix_time) FROM tweet_titles)

"""


In [90]:
try:
    df = pd.read_sql_query(q_extract, engine)
    print(df.info())
except:
    print("failed to load data")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115681 entries, 0 to 115680
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   S_No       115681 non-null  int64 
 1   user_id    115681 non-null  int64 
 2   username   115681 non-null  object
 3   tweet      115673 non-null  object
 4   unix_time  115681 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 4.4+ MB
None


In [91]:
#dropping null values
df = df.dropna()
print("#after dropping null \n", df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115673 entries, 0 to 115679
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   S_No       115673 non-null  int64 
 1   user_id    115673 non-null  int64 
 2   username   115673 non-null  object
 3   tweet      115673 non-null  object
 4   unix_time  115673 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 5.3+ MB
#after dropping null 
 None


In [92]:
# # dropping tweets not in desired format
# similar_sentences = df[~df['tweet'].str.contains("rated")]
# similar_sentences

In [93]:
#extracting titles,year and ratings from tweets
df[['title','year','rating']] = df['tweet'].str.extract(r'rated (.*) \((.*)\) (.*)')
print("#after extracting strings", df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115673 entries, 0 to 115679
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   S_No       115673 non-null  int64 
 1   user_id    115673 non-null  int64 
 2   username   115673 non-null  object
 3   tweet      115673 non-null  object
 4   unix_time  115673 non-null  int64 
 5   title      114736 non-null  object
 6   year       114736 non-null  object
 7   rating     114736 non-null  object
dtypes: int64(3), object(5)
memory usage: 7.9+ MB
#after extracting strings None


In [94]:
df[df['title'].isnull()]


,S_No,user_id,username,tweet,unix_time,title,year,rating
56,57,14248401,bf79,I rated Bullet Train 8/10,1674659099,NaN,NaN,NaN
103,104,14248401,bf79,I rated Good Night Oppy 7/10,1674532317,NaN,NaN,NaN
258,259,833012252,KennedyMike4,I rated The Deep House 7/10,1674445603,NaN,NaN,NaN
273,274,432023011,Makavelimademe,#VELMA has been considered the #1 WORST rated ...,1674440845,NaN,NaN,NaN
316,317,1518968288335835136,HorrorLovesBeth,I almost never trust an IMDb score. I have lov...,1674422797,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
115410,117438,1539480739,robert_cashin,#GetCarter (1971),1604543364,NaN,NaN,NaN
115452,117480,58084897,tinashiba48,I rated Lucifer 「ルシファー」(2016– ) 8/10,1604512879,NaN,NaN,NaN
115517,117545,1589202139859009538,StevenK_007,I rated The Last Movie Star 2017) 9/10,1674977497,NaN,NaN,NaN
115538,117566,833012252,KennedyMike4,I rated Broadcast Signal Intrusion 6.5/10,1674970461,NaN,NaN,NaN


In [95]:
#removing null titles
df_2 = df[df['title'].isnull()]
df = df[~df['S_No'].isin(df_2['S_No'])]
#df[df['title'].isnull()]

In [96]:
df_1 = df[df['year'].str.len()>4]
df = df[~df['S_No'].isin(df_1['S_No'])]
print("#after removing tv-shows", df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83038 entries, 0 to 115679
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   S_No       83038 non-null  int64 
 1   user_id    83038 non-null  int64 
 2   username   83038 non-null  object
 3   tweet      83038 non-null  object
 4   unix_time  83038 non-null  int64 
 5   title      83038 non-null  object
 6   year       83038 non-null  object
 7   rating     83038 non-null  object
dtypes: int64(3), object(5)
memory usage: 5.7+ MB
#after removing tv-shows None


In [97]:
# df = df.drop('S_No', axis=1)
# df.info()

In [104]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)

now = 2023-01-29 20:45:40.901331
date and time = 29/01/2023 20:45:40
